<a href="https://colab.research.google.com/github/jlopetegui98/NER-ClinicalTrials-Eligibility-Criteria/blob/main/Roberta%2BLLM/mistral7b-full-evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# uncomment if working in colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# uncomment if using colab
!pip install -q -U datasets
!pip install seqeval
!pip install -U torch
!pip install -q -U evaluate
!pip install -q -U git+https://github.com/huggingface/transformers.git
# !pip install -q -U transformers
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
# !pip install -q -U accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TextGenerationPipeline
import os
from utils import *

In [3]:
import pandas as pd
from datasets import Dataset, DatasetDict, load_dataset

In [4]:
import re

In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [6]:
# dict for the entities (entity to int value)
simple_ent = {"Condition", "Value", "Drug", "Procedure", "Measurement", "Temporal", "Observation", "Person", "Device"}
sel_ent = {
    "O": 0,
    "B-Condition": 1,
    "I-Condition": 2,
    "B-Value": 3,
    "I-Value": 4,
    "B-Drug": 5,
    "I-Drug": 6,
    "B-Procedure": 7,
    "I-Procedure": 8,
    "B-Measurement": 9,
    "I-Measurement": 10,
    "B-Temporal": 11,
    "I-Temporal": 12,
    "B-Observation": 13,
    "I-Observation": 14,
    "B-Person": 15,
    "I-Person": 16,
    "B-Device": 17,
    "I-Device": 18
}

entities_list = list(sel_ent.keys())
sel_ent_inv = {v: k for k, v in sel_ent.items()}

In [7]:
root = '..'
root = './drive/MyDrive/TER-LISN-2024'
data_path = f'{root}/data'
models_path = f'{root}/models'

In [8]:
model_name = "mistralai/Mistral-7B-v0.1"

In [9]:
# Load base model(Mistral 7B)
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
   model_name,
    quantization_config=bnb_config,
    device_map={"": 0}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
# import tokenizer for mistral-7B
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.padding_side = 'left'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = False
tokenizer.add_bos_token, tokenizer.add_eos_token

(True, False)

In [11]:
pipe  = TextGenerationPipeline(model = model, tokenizer = tokenizer)

In [12]:
dataset = load_dataset('JavierLopetegui/chia_v1')

In [13]:
# for each sentence save the text
def generate_sentences_from_tokens(sentences):
    texts_sentences = []
    sentences_tokens = sentences['tokens']
    for sentence in sentences_tokens:
        sent_text = " ".join(sentence)
        texts_sentences.append(sent_text)
    sentences['text'] = texts_sentences
    return sentences

In [14]:
def build_prompts(sentences, prompt_type=2):
    sentences_prompts = []
    for sent in sentences['text']:
        prompt = build_prompt(sent, prompt_type)
        sentences_prompts.append(prompt)
    sentences['prompt'] = sentences_prompts
    return sentences

In [15]:
dataset = dataset.map(lambda x: generate_sentences_from_tokens(x), batched = True)
dataset_prompt2 = dataset.map(lambda x: build_prompts(x, prompt_type=2), batched = True)

In [16]:
test_dataset_p2 = dataset_prompt2['test']

In [17]:
# keep just the prompt column
test_dataset_p2 = test_dataset_p2.remove_columns(['tokens', 'text', 'ner_tags', 'file'])

In [ ]:
# data_loader_p2 = DataLoader(test_dataset_p2, batch_size=4, shuffle=False)

In [ ]:
# generated_sentences_p2 = pipe(batch['prompt'], max_new_tokens = 500, return_full_text = False, handle_long_generation = "hole"))

In [18]:
from tqdm import tqdm

In [59]:
print(test_dataset_p2['prompt'][0])

I am working on a named entity recognition problem, in the context of clinical
    trials eligibility criteria. I will show you the list of entities:
    - Condition
    - Value
    - Drug
    - Procedure
    - Measurement
    - Temporal
    - Observation
    - Person
    - Device

    Your task consists in annotate the named entities in a given sentence in the format I will explain you.
    I will explain you with some examples:

    Example 1:
    Input: Patients who have received prior chemotherapy for unresectable disease.
    Output: Patients who have received prior <Procedure>chemotherapy</Procedure> for <Condition>unresectable disease</Condition>.

    Example 2:
    Input: Patients with any other severe concurrent disease, which in the judgment of the investigator, would make the patient inappropriate for entry into this study.
    Ouput: Patients with any other severe <Condition>concurrent disease</Condition>, which in the judgment of the investigator, would make the patient i

In [88]:
generated_sentences_p2 = []
for sentence in tqdm(test_dataset_p2['prompt']):
    sentence += '\noutput: '
    output = pipe(sentence, max_new_tokens = 500, return_full_text = False, handle_long_generation = "hole")[0]['generated_text']
    output = output.split('\n')[0]
    generated_sentences_p2.append(output)

100%|██████████| 1307/1307 [15:16:55<00:00, 42.09s/it]


In [89]:
len(generated_sentences_p2)

1307

In [90]:
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')

In [91]:
# tokenize and align the labels in the dataset
def tokenize_and_align_labels(sentence, flag = 'I'):
    """
    Tokenize the sentence and align the labels
    inputs:
        sentence: dict, the sentence from the dataset
        flag: str, the flag to indicate how to deal with the labels for subwords
            - 'I': use the label of the first subword for all subwords but as intermediate (I-ENT)
            - 'B': use the label of the first subword for all subwords as beginning (B-ENT)
            - None: use -100 for subwords
    outputs:
        tokenized_sentence: dict, the tokenized sentence now with a field for the labels
    """
    tokenized_sentence = tokenizer(sentence['tokens'], is_split_into_words=True, truncation=True)

    labels = []
    for i, labels_s in enumerate(sentence['ner_tags']):
        word_ids = tokenized_sentence.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # if the word_idx is None, assign -100
            if word_idx is None:
                label_ids.append(-100)
            # if it is a new word, assign the corresponding label
            elif word_idx != previous_word_idx:
                label_ids.append(labels_s[word_idx])
            # if it is the same word, check the flag to assign
            else:
                if flag == 'I':
                    if entities_list[labels_s[word_idx]].startswith('I'):
                      label_ids.append(labels_s[word_idx])
                    else:
                      label_ids.append(labels_s[word_idx] + 1)
                elif flag == 'B':
                    label_ids.append(labels_s[word_idx])
                elif flag == None:
                    label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_sentence['labels'] = labels
    return tokenized_sentence

**Standarizing true annotations**

In [92]:
def parse_ann2bio(sentence, pattern, pattern1, pattern2):
    # if sentence[-1] == "\n":
    #     sentence = sentence[:-2] # remove the \n and a final point wrongly added
    # else:
    #     sentence = sentence[:-1] # remove the final point wrongly added

    # find the entities
    occurrences = re.finditer(pattern, sentence)
    indexes = [(match.start(), match.end()) for match in occurrences]

    annotation = []
    i = 0


    # create the bio list
    for beg, end in indexes:
        if beg > i:
            annotation.extend([(word, "O") for word in sentence[i:beg].split()])
        entity = sentence[beg:end]
        entity_name = re.search(pattern1, entity).group(1)
        entity = entity.replace(f'<{entity_name}>', "").replace(f'</{entity_name}>', "")
        split_entity = entity.split()
        annotation.append((split_entity[0], "B-" + entity_name))
        annotation.extend([(word, "I-" + entity_name) for word in split_entity[1:]])
        i = end
    annotation.extend([(word, "O") for word in sentence[i:].split()])

    # check punctuation sign in tokens and put them as individual tokens
    ps = r'(\.|\,|\:|\;|\!|\?|\-|\(|\)|\[|\]|\{|\}|\")'
    new_annotation = []
    for i,(word, tag) in enumerate(annotation):
        if re.search(ps, word):
            # find the ocurrences of the punctuation signs
            occurrences = re.finditer(ps, word)
            indexes = [(match.start(), match.end()) for match in occurrences]
            # create the new tokens
            last = 0
            for j, (beg, end) in enumerate(indexes):
                if beg > last:
                    new_annotation.append((word[last:beg], tag))
                if tag != "O":
                    label = f'I-{tag.split("-")[1]}'
                else:
                    label = "O"
                if end < len(word) or (i < len(annotation) - 1 and annotation[i+1][1] == label):
                    new_annotation.append((word[beg:end], label))
                else:
                    new_annotation.append((word[beg:end], 'O'))
                last = end
            if last < len(word):
                new_annotation.append((word[last:], label))

        else:
            new_annotation.append((word, tag))


    return new_annotation

In [93]:
new_true_annotations = []
for sent in dataset['test']:
    annotation = []
    for word, tag in zip(sent['tokens'], sent['ner_tags']):
        annotation.append((word, entities_list[tag]))
    new_annotation = []
    ps = r'(\.|\,|\:|\;|\!|\?|\-|\(|\)|\[|\]|\{|\}|\")'
    for i,(word, tag) in enumerate(annotation):
        if re.search(ps, word):
            # find the ocurrences of the punctuation signs
            occurrences = re.finditer(ps, word)
            indexes = [(match.start(), match.end()) for match in occurrences]
            # create the new tokens
            last = 0
            for j, (beg, end) in enumerate(indexes):
                if beg > last:
                    new_annotation.append((word[last:beg], tag))
                if tag != "O":
                    label = f'I-{tag.split("-")[1]}'
                else:
                    label = "O"
                if end < len(word) or (i < len(annotation) - 1 and annotation[i+1][1] == label):
                    new_annotation.append((word[beg:end], label))
                else:
                    new_annotation.append((word[beg:end], 'O'))
                last = end
            if last < len(word):
                new_annotation.append((word[last:], label))
        else:
            new_annotation.append((word, tag))
    new_true_annotations.append(new_annotation)
len(new_true_annotations)

1307

In [94]:
true_annotations = []
for sent in new_true_annotations:
    dicc_sent = {"tokens":[], "ner_tags":[]}
    for word, tag in sent:
        dicc_sent["tokens"].append(word)
        dicc_sent["ner_tags"].append(sel_ent[tag])
    true_annotations.append(dicc_sent)
len(true_annotations)

1307

In [95]:
true_df = pd.DataFrame(true_annotations)
true_ann_dataset = Dataset.from_pandas(true_df)

In [96]:
true_ann_dataset = true_ann_dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/1307 [00:00<?, ? examples/s]

In [145]:
def get_labels(p):
    predictions, labels = p
    # Remove ignored index (special tokens)
    # predictions = [
    #     [entities_list[p] for (p, l) in zip(prediction, label) if l != -100]
    #     for prediction, label in zip(predictions, labels)
    # ]
    predictions_ = []
    for (pred,lab) in zip(predictions, labels):
        predictions_.append([])
        for (p,l) in zip(pred, lab):
          if l != -100:
            if p == -100:
              predictions_[-1].append(entities_list[0])
            else:
              predictions_[-1].append(entities_list[p])

    labels = [
        [entities_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    return predictions_, labels

In [98]:
from eval_file import *

**Evaluating prompt 2**

In [99]:
pattern1 = r'<(Person|Condition|Value|Drug|Procedure|Measurement|Temporal|Observation|Device)>'
pattern2 = r'</(Person|Condition|Value|Drug|Procedure|Measurement|Temporal|Observation|Device)>'
pattern = f'{pattern1}.*?{pattern2}'

In [100]:
new_p2_annotations = []
for sent in generated_sentences_p2:
    annotation = parse_ann2bio(sent, pattern, pattern1, pattern2)
    new_p2_annotations.append(annotation)
len(new_p2_annotations)

1307

In [101]:
p2_annotations = []
for sent in new_p2_annotations:
    dicc_sent = {"tokens":[], "ner_tags":[]}
    for word, tag in sent:
        dicc_sent["tokens"].append(word)
        dicc_sent["ner_tags"].append(sel_ent[tag])
    p2_annotations.append(dicc_sent)
len(p2_annotations)

1307

In [102]:
p2_df = pd.DataFrame(p2_annotations)
p2_dataset = Dataset.from_pandas(p2_df)

In [103]:
p2_dataset = p2_dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/1307 [00:00<?, ? examples/s]

In [104]:
# keep just sentences with the same length
sentences_to_evaluate_p2 = []
sentences_to_evaluate_true = []

for i in range(len(p2_dataset)):
    # keep the min len among the true and annotated sentence
    l = min(len(p2_dataset['labels'][i]), len(true_ann_dataset['labels'][i]))
    sentences_to_evaluate_p2.append(p2_dataset['labels'][i][:l])
    sentences_to_evaluate_true.append(true_ann_dataset['labels'][i][:l])
    # if len(p2_dataset['labels'][i]) != len(true_ann_dataset['labels'][i]):
    #     print(p2_dataset['tokens'][i])
    #     print(true_ann_dataset['tokens'][i])
    #     print(get_labels(([p2_dataset['labels'][i]], [true_ann_dataset['labels'][i]])))
    #     # print(true_ann_dataset['labels'][i])
    #     # sentences_to_evaluate_p2.append(p2_dataset['labels'][i])
    #     # sentences_to_evaluate_true.append(true_ann_dataset['labels'][i])

print(len(sentences_to_evaluate_p2)/len(p2_dataset))

1.0


In [105]:
evaluator = BioEval()

In [113]:
# sentences_to_evaluate_p2[:2]
for sentence in sentences_to_evaluate_p2:
  if len(sentence) == 0:
    print('len000')


In [146]:
pred_labels, true_labels = get_labels((sentences_to_evaluate_p2, sentences_to_evaluate_true))

In [140]:
evaluator.evaluate_annotations(true_labels, pred_labels, do_lower=True)

In [141]:
evaluator.performance

{'overall': {'acc': 0.6039,
  'strict': {'precision': 0.539911466514351,
   'recall': 0.5364642451759364,
   'f_score': 0.5381823357768131},
  'relax': {'precision': 0.6547194059688705,
   'recall': 0.6505391600454029,
   'f_score': 0.652622589139563}},
 'category': {'strict': {'condition': {'precision': 0.6026216555934638,
    'recall': 0.6669316375198728,
    'f_score': 0.6331478162437506},
   'measurement': {'precision': 0.14698162729658792,
    'recall': 0.19310344827586207,
    'f_score': 0.16691505216095381},
   'drug': {'precision': 0.562358276643991,
    'recall': 0.5598194130925508,
    'f_score': 0.5610859728506786},
   'procedure': {'precision': 0.3992248062015504,
    'recall': 0.329073482428115,
    'f_score': 0.36077057793345},
   'temporal': {'precision': 0.030303030303030304,
    'recall': 0.003367003367003367,
    'f_score': 0.006060606060606061},
   'person': {'precision': 0.053763440860215055,
    'recall': 0.03759398496240601,
    'f_score': 0.04424778761061946},
  

In [143]:
evaluator.save_evaluation('eval_paper_full_generative.json')

In [142]:
evaluator.get_counts()

{'expect': {'overall': 7048,
  'condition': 5032,
  'person': 133,
  'value': 351,
  'observation': 166,
  'drug': 443,
  'measurement': 290,
  'procedure': 313,
  'temporal': 297,
  'device': 23},
 'prediction': {'strict': {'overall': {'total': 7003,
    'true': 3781,
    'false': 3222},
   'condition': {'total': 5569, 'true': 3356, 'false': 2213},
   'measurement': {'total': 381, 'true': 56, 'false': 325},
   'drug': {'total': 441, 'true': 248, 'false': 193},
   'procedure': {'total': 258, 'true': 103, 'false': 155},
   'temporal': {'total': 33, 'true': 1, 'false': 32},
   'person': {'total': 93, 'true': 5, 'false': 88},
   'observation': {'total': 185, 'true': 3, 'false': 182},
   'value': {'total': 18, 'true': 1, 'false': 17},
   'device': {'total': 25, 'true': 8, 'false': 17}},
  'relax': {'overall': {'total': 7003, 'true': 4585, 'false': 2418},
   'condition': {'total': 5569, 'true': 3977, 'false': 1592},
   'measurement': {'total': 381, 'true': 96, 'false': 285},
   'drug': {'to

In [144]:
evaluator.save_evaluation('drive/MyDrive/TER-LISN-2024/eval_paper_full_generative.json')